In [2]:
import torch

torch.device("cpu")

False

In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np

# installed opencv via pip in pip install opencv-python
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
from detectron2.data.datasets import register_coco_instances

TRAINING_IMG_DIR = "./images/All_Images"
TRAINING_ANNOTATION_DIR = "./images/labels_116Images_2021-12-01-06-45-38.json"

register_coco_instances("my_dataset_train", {}, TRAINING_ANNOTATION_DIR, TRAINING_IMG_DIR)

In [5]:
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 2000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
cfg.OUTPUT_DIR = "./images"

cfg.MODEL.WEIGHTS = "./model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.DEVICE='cpu'

predictor = DefaultPredictor(cfg)

# Initialize visualizer
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
v = VideoVisualizer(my_dataset_train_metadata, ColorMode.IMAGE)

In [ ]:
video_path = r"D:\West_Village_Waste_Footage\11-16-2021\Front-Trash-Can\2021\11\16\Val7.h264"
MASKRCNN_MODEL = r"C:\Users\IvanZou\OneDrive\Documents\OpenCV_Python3\Waste_Detection\weights"

cap = cv2.VideoCapture(video_path)

if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    res=(int(width), int(height))
    videowriter = cv2.VideoWriter('MyOutputVid.avi', cv2.VideoWriter_fourcc('I','4','2','0'), fps, res)

    frame = None
    counter = 0

    while True:
        try:
            is_success, frame = cap.read()
        except cv2.error:
            continue

        if not is_success:
            break

        # Write image to video writer
        outputs = predictor(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        visualization = v.draw_instance_predictions(frame, outputs["instances"])
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)
        videowriter.write(visualization)

        if counter > 2000: 
          break

        counter = counter + 1

    videowriter.release() 

    # Show last image

    cv2_imshow(visualization)

cap.release()